# PyTerrier Notebook for Full-Rank Submissions

This notebook serves as a baseline full-rank submission for [TIRA](https://tira.io)/[TIREx](https://tira.io/tirex) that builds a PyTerrier index and subsequently creates a run with BM25.

### Step 1: Ensure Libraries are Imported

In [1]:
import os

# Detect if we are in the TIRA sandbox
# Install the required dependencies if we are not in the sandbox.
if 'TIRA_DATASET_ID' not in os.environ:
    !python -m pip install --upgrade pip
    !pip3 install  python-terrier tira==0.0.88 ir_datasets
    !pip3 install -q python-terrier
    !pip3 install -q --upgrade git+https://github.com/terrierteam/pyterrier_t5.git
else:
    print('We are in the TIRA sandbox.')

from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run

# PyTerrier must be imported after the call to ensure_pyterrier_is_loaded in TIRA.
import pyterrier as pt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 19.3 MB/s eta 0:00:0000:01
  Attempting uninstall: pip
    Found existing installation: pip 23.3.1
    Uninstalling pip-23.3.1:
      Successfully uninstalled pip-23.3.1
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.6/107.6 kB 2.0 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 1.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
    

/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Ensure Pyterrier integration is loaded

In [2]:
ensure_pyterrier_is_loaded()

Due to execution in TIRA, I have patched ir_datasets to always return the single input dataset mounted to the sandbox.
Start PyTerrier with version=5.7, helper_version=0.0.7, no_download=True
terrier-assemblies 5.7 jar-with-dependencies not found, downloading to /home/codespace/.pyterrier...
Done
terrier-python-helper 0.0.7 jar not found, downloading to /home/codespace/.pyterrier...
Done
terrier-prf -SNAPSHOT jar not found, downloading to /home/codespace/.pyterrier...
Done


PyTerrier 0.10.0 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


### Step 2: Load data, create index

In [11]:
# dataset_name = 'validation-20231104-training'
dataset_name = 'jena-topics-20231026-test'
dataset = pt.get_dataset('irds:ir-lab-jena-leipzig-wise-2023/'+dataset_name)
topics = dataset.get_topics(variant="title")
qrels = dataset.get_qrels()

from pathlib import Path
index_loc = "./index"
if not (Path(index_loc) / "data.properties").exists():
    indexer = pt.IterDictIndexer(index_loc)
    indexref = indexer.index(dataset.get_corpus_iter())
else:
    indexref = pt.IndexFactory.of(index_loc)

Load ir_dataset "ir-lab-jena-leipzig-wise-2023/jena-topics-20231026-test" from tira.
No settings given in /home/codespace/.tira/.tira-settings.json. I will use defaults.
No settings given in /home/codespace/.tira/.tira-settings.json. I will use defaults.


No settings given in /home/codespace/.tira/.tira-settings.json. I will use defaults.
No settings given in /home/codespace/.tira/.tira-settings.json. I will use defaults.


In [12]:
from tira.rest_api_client import Client
tira_client = Client()

No settings given in /home/codespace/.tira/.tira-settings.json. I will use defaults.
No settings given in /home/codespace/.tira/.tira-settings.json. I will use defaults.


In [13]:
bm25_bo1_pl2_max = tira_client.pt.from_retriever_submission('ir-lab-jena-leipzig-wise-2023/galapagos-tortoise/mild-duck', dataset_name) 
max_run = bm25_bo1_pl2_max.transform(topics)

#### Secondly, rerank with mean passage aggregation.

In [14]:
bm25_bo1_pl2_mean = tira_client.pt.from_retriever_submission('ir-lab-jena-leipzig-wise-2023/galapagos-tortoise/poky-claim', dataset_name)
mean_run = bm25_bo1_pl2_mean.transform(topics)

In [15]:
bm25_bo1_pl2_kmax = tira_client.pt.from_retriever_submission('ir-lab-jena-leipzig-wise-2023/galapagos-tortoise/edible-status', dataset_name)
kmax_run = bm25_bo1_pl2_kmax.transform(topics)

In [18]:
experiment = pt.Experiment(
        [
            bm25_bo1_pl2_max,
            bm25_bo1_pl2_mean,
            bm25_bo1_pl2_kmax,
        ],
        topics,
        qrels,
        eval_metrics=["ndcg_cut_5", "ndcg"],
        names=[
            "max passage",
            "mean passage",
            "best kmax passage",
        ],
        # perquery=True
    )

hypo1 = pt.Experiment(
    [bm25_bo1_pl2_max, bm25_bo1_pl2_mean],
    topics,
    qrels,
    eval_metrics=["ndcg_cut_5", "ndcg"],
    names=["max passage", "mean passage"],
    baseline = 0
)

hypo2 = pt.Experiment(
    [bm25_bo1_pl2_kmax, bm25_bo1_pl2_max, bm25_bo1_pl2_mean],
    topics,
    qrels,
    eval_metrics=["ndcg_cut_5", "ndcg"],
    names=["best kmax passage", "max passage", "mean passage"],
    baseline = 0
)

experiment

,name,ndcg_cut_5,ndcg
0,max passage,0.268758,0.373611
1,mean passage,0.255592,0.369422
2,best kmax passage,0.283274,0.385448


### Persist experiment and tests.

In [19]:
hypo1

,name,ndcg,ndcg_cut_5,ndcg +,ndcg -,ndcg p-value,ndcg_cut_5 +,ndcg_cut_5 -,ndcg_cut_5 p-value
0,max passage,0.373611,0.268758,NaN,NaN,NaN,NaN,NaN,NaN
1,mean passage,0.369422,0.255592,19.0,15.0,0.819999,12.0,14.0,0.630029


In [20]:
hypo2

,name,ndcg,ndcg_cut_5,ndcg +,ndcg -,ndcg p-value,ndcg_cut_5 +,ndcg_cut_5 -,ndcg_cut_5 p-value
0,best kmax passage,0.385448,0.283274,NaN,NaN,NaN,NaN,NaN,NaN
1,max passage,0.373611,0.268758,12.0,20.0,0.147035,7.0,13.0,0.266367
2,mean passage,0.369422,0.255592,14.0,19.0,0.321049,9.0,14.0,0.311403


In [21]:
with open("results.txt", "wt") as file:
    file.write("Experiment:\n\n")
    experiment_string = experiment.to_string(header=True, index=False)
    file.write(experiment_string + "\n\n")
    # ...
    file.write("Results for Hypothesis 1:\n\n")
    file.write("Significance test:\n")
    hypo1_string = hypo1.to_string(header=True, index=False)
    file.write(hypo1_string + "\n\n")
    # ...
    file.write("Results for Hypothesis 2:\n\n")
    file.write("Significance test:\n")
    hypo2_string = hypo2.to_string(header=True, index=False)
    file.write(hypo2_string)
    # ...